In [1]:
import torch
from torchtext.datasets import IMDB
from torch.utils.data import DataLoader

In [22]:
data_iter = IMDB(split='train')

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

toker = get_tokenizer("basic_english")
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for _,text in data_iter:
        yield toker(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=['unk'])
vocab.set_default_index(vocab['unk'])

AttributeError: ignored

In [ ]:
vocab(toker("I like ice"))

[12, 44, 2065]

In [ ]:
text_pipeline = lambda x: vocab(toker(x))
label_pipeline = lambda x: int(x) - 1

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [ ]:
train_iter = IMDB(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [ ]:
num_class = len(set([label for (label,text) in train_iter]))

In [ ]:
num_class

2

In [ ]:
len(vocab)

100683

In [ ]:
import torch.nn as nn